# **Lab Task 10 (Skeleton Code)**

In [27]:
import numpy as np
import random

# Initialize an empty 6x7 Connect-4 board
def init_board():
    """
    Initialize the Connect-4 board.
    The board is represented as a 6x7 grid where 0 represents an empty space,
    1 represents the human player, and 2 represents the AI player.
    """
    grid = []
    for i in range(6):
        row = []
        for j in range(7):
            row.append(0)
        grid.append(row)
    
    return grid



In [41]:

# Print the board in a readable format
def display_board(board):
    """
    Print the Connect-4 board to the console in a readable format.
    The board is printed from the bottom row to the top.
    """
    for row in board:
        for i in row:
            print(i, end='\t')
        print()
        
display_board(init_board())






0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	


In [29]:
# Get a list of valid columns where a move can be made
def available_columns(board):
    """
    Generate a list of valid columns where a piece can be dropped.
    A column is valid if the topmost space is empty.
    """
    valid = []
    for i in range(7):
        if(board[0][i] == 0):
            valid.append(i)
    return valid




In [30]:
# Drop a piece into the board
def drop_piece(board, column, player):
    """
    Drop the piece (either 1 for human or 2 for AI) into the specified column.
    The piece should fall to the first available empty row in that column.
    """
    for i in range(5, 0, -1):
        if(board[i][column] == 0):
            board[i][column] = player
            break
        



In [31]:
# Check for a winning condition
def check_winner(board, player):
    """
    Check if the specified player (1 or 2) has won the game.
    A player wins if they have four of their pieces in a row either:
        - Horizontally
        - Vertically
        - Diagonally (both left and right)
    """
    ROWS = 6
    COLS = 7

    for row in range(ROWS):
        for col in range(COLS - 3):
            if all(board[row][col + i] == player for i in range(4)):
                return True

    for col in range(COLS):
        for row in range(ROWS - 3):
            if all(board[row + i][col] == player for i in range(4)):
                return True

    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            if all(board[row + i][col + i] == player for i in range(4)):
                return True

    for row in range(3, ROWS):
        for col in range(COLS - 3):
            if all(board[row - i][col + i] == player for i in range(4)):
                return True

    return False




In [43]:
import math
def evaluate_window(window, player):
    score = 0
    opp = 2 if player == 1 else 1
    count_player = window.count(player)
    count_empty = window.count(0)
    count_opp = window.count(opp)

    if count_player == 4:
        score += 100
    elif count_player == 3 and count_empty == 1:
        score += 10
    elif count_player == 2 and count_empty == 2:
        score += 5

    if count_opp == 3 and count_empty == 1:
        score -= 8
    return score

def score_position(board, player):
    """
    Evaluate the board state for the given player.
    If the player has won, return a high positive value.
    If the opponent has won, return a high negative value.
    Otherwise, return a heuristic evaluation score.
    """
    ROWS = 6
    COLS = 7
    score = 0
    opponent = 2 if player == 1 else 1

    # Terminal state check:
    if check_winner(board, player):
        return math.inf
    elif check_winner(board, opponent):
        return -math.inf

    # Score center column for advantage
    center_array = [board[r][COLS // 2] for r in range(ROWS)]
    center_count = center_array.count(player)
    score += center_count * 6

    # Score horizontal windows
    for row in range(ROWS):
        for col in range(COLS - 3):
            window = [board[row][col + i] for i in range(4)]
            score += evaluate_window(window, player)

    # Score vertical windows
    for col in range(COLS):
        for row in range(ROWS - 3):
            window = [board[row + i][col] for i in range(4)]
            score += evaluate_window(window, player)

    # Score positive slope diagonals
    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            window = [board[row + i][col + i] for i in range(4)]
            score += evaluate_window(window, player)

    # Score negative slope diagonals
    for row in range(3, ROWS):
        for col in range(COLS - 3):
            window = [board[row - i][col + i] for i in range(4)]
            score += evaluate_window(window, player)

    return score

In [36]:
import math
def minimax(board, depth, alpha, beta, maximizing_player):
    valid_columns = available_columns(board)
    
    # Terminal check: depth reached or game over
    if depth == 0 or check_winner(board, 1) or check_winner(board, 2) or len(valid_columns) == 0:
        return None, score_position(board, 1)  # Score from AI's (player 1) perspective

    if maximizing_player:
        value = -math.inf
        best_column = random.choice(valid_columns)
        for col in valid_columns:
            new_board = [row[:] for row in board]  # Copy board
            drop_piece(new_board, col, 1)           # AI's move (player 1)
            new_score = minimax(new_board, depth - 1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break  # Prune branch
        return best_column, value
    else:
        value = math.inf
        best_column = random.choice(valid_columns)
        for col in valid_columns:
            new_board = [row[:] for row in board]  # Copy board
            drop_piece(new_board, col, 2)           # Human's move (player 2)
            new_score = minimax(new_board, depth - 1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_column = col
            beta = min(beta, value)
            if alpha >= beta:
                break  # Prune branch
        return best_column, value



In [37]:
def best_move(board):
    DEPTH = 4  # Choose desired search depth
    column, _ = minimax(board, DEPTH, -math.inf, math.inf, True)
    return column

In [38]:
def human_move(board):
    """
    Allow the human player to choose a valid move (column number).
    This function will prompt the user until they enter a valid column (0-6) that is not full.
    It returns the chosen column.
    """
    valid = available_columns(board)  # Your function that returns available columns
    
    # Prompt until we get a valid column
    while True:
        try:
            col = int(input("Choose a column (0-6): "))
            if col in valid:
                return col
            else:
                print("Column is either full or out of range. Valid columns are:", valid)
        except ValueError:
            print("Invalid input. Please enter a number between 0 and 6.")


In [39]:
def play_game():
    """
    Run the Connect-4 game simulation: the human (player 2) and AI (player 1) alternate moves.
    """
    board = init_board()
    game_over = False
    turn = 0  # 0: Human's turn, 1: AI's turn

    while not game_over:
        display_board(board)
        
        if turn == 0:
            # Human's turn (player 2)
            col = human_move(board)
            drop_piece(board, col, 2)
            if check_winner(board, 2):
                display_board(board)
                print("Congratulations! You win!")
                game_over = True
        else:
            # AI's turn (player 1)
            print("AI is making a move...")
            col = best_move(board)
            drop_piece(board, col, 1)
            if check_winner(board, 1):
                display_board(board)
                print("AI wins! Better luck next time.")
                game_over = True
        
        # Check for tie
        if not game_over and len(available_columns(board)) == 0:
            display_board(board)
            print("It's a tie!")
            game_over = True
        
        turn = 1 - turn  # Switch turns

In [42]:
# Start the game
if __name__ == "__main__":
    play_game()


0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
2	0	0	0	0	0	0	
AI is making a move...
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
2	0	0	1	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
2	0	0	1	2	0	0	
AI is making a move...
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	1	0	0	0	
2	0	0	1	2	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	1	2	0	0	
2	0	0	1	2	0	0	
AI is making a move...
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	1	0	0	0	
0	0	0	1	2	0	0	
2	0	0	1	2	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	2	0	0	0	
0	0	0	1	0	0	0	
0	0	0	1	2	0	0	
2	0	0	1	2	0	0	
AI is making a move...
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	2	0	0	0	
0	0	0	1	1	0	0	
0	0	0	1	2	0	0	
2	0	0	1	2	0	0	
0	0	0	0	0	0	0	
0	0	0	0	0	0	0	
0	0	0	2	2	0	0	
0	0	0	1	1	0	0	
0	0	0	1	2	0	0	
2	0	0	1	2	0	0	
AI is ma